<a href="https://colab.research.google.com/github/miltiadiss/Data-Mining/blob/main/question_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**2.3 Bayesian Network**

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, HillClimbSearch, BicScore
from pgmpy.inference import VariableElimination
from sklearn.preprocessing import KBinsDiscretizer

def discretize_columns(df, bins):
    cols_to_discretize = [col for col in df.columns if col not in ['timestamp', 'label']]
    discretizer = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='uniform')
    df[cols_to_discretize] = discretizer.fit_transform(df[cols_to_discretize])
    return df

# Λίστες για την αποθήκευση των μετρικών από κάθε αρχείο
bn_accuracies = []  # Ακρίβεια για κάθε αρχείο
bn_precisions = []  # Precision για κάθε αρχείο
bn_recalls = []     # Recall για κάθε αρχείο
bn_f1_scores = []   # F1-score για κάθε αρχείο

# Επανάληψη μέσω των αρχείων στο φάκελο
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        # Φόρτωση του CSV αρχείου
        df = pd.read_csv(os.path.join(path, filename))

        # Δημιουργία των καθυστερήσεων
        df = create_lags(df, 10)

        # Διακριτοποίηση των χαρακτηριστικών
        df = discretize_columns(df, bins=3)

        # Διαίρεση σε train και test σύνολα
        train_size = int(0.8 * len(df))
        train_df = df.iloc[:train_size]
        test_df = df.iloc[train_size:]

        X_train = train_df.drop(['timestamp', 'label'], axis=1)
        y_train = train_df['label']
        X_test = test_df.drop(['timestamp', 'label'], axis=1)
        y_test = test_df['label']

        # Συνδυασμός X_train και y_train για δημιουργία DataFrame για εκπαίδευση του Bayesian Network
        train_df = pd.concat([X_train, y_train], axis=1)

        # Δημιουργία δομής του Bayesian Network με χρήση του αλγορίθμου HillClimbSearch και της μετρικής BIC
        hc = HillClimbSearch(train_df)
        best_model = hc.estimate(scoring_method=BicScore(train_df))

        # Δημιουργία του μοντέλου Bayesian Network με βάση τις βέλτιστες συνδέσεις που προέκυψαν από τον αλγόριθμο HillClimbSearch
        model = BayesianNetwork(best_model.edges())
        model.fit(train_df, estimator=MaximumLikelihoodEstimator)

        # Απόδοση του μοντέλου
        infer = VariableElimination(model)
        y_pred = []

        for _, row in X_test.iterrows():
            evidence = row.to_dict()
            query_result = infer.map_query(variables=['label'], evidence=evidence)
            y_pred.append(query_result['label'])

        # Υπολογίζουμε τις μετρικες και τις προσθέτουμε στις λίστες
        accuracy = accuracy_score(y_test, y_pred)
        bn_accuracies.append(accuracy)
        precision = precision_score(y_test, y_pred)
        bn_precisions.append(precision)
        recall = recall_score(y_test, y_pred)
        bn_recalls.append(recall)
        f1 = f1_score(y_test, y_pred)
        bn_f1_scores.append(f1)

        # Εκτύπωση των μετρικών για κάθε συμμετέχοντα
        base_name = os.path.splitext(filename)[0]  # Χωρίς κατάληξη
        print(f"Metrics for Participant {base_name}:")
        print(f"  Accuracy: {accuracy}")
        print(f"  Precision: {precision}")
        print(f"  Recall: {recall}")
        print(f"  F1-Score: {f1}")

        # Υπολογισμός και εμφάνιση του confusion matrix
        unique_labels = sorted(df['label'].unique())
        cm = confusion_matrix(y_test, y_pred, labels=unique_labels)
        plt.figure(figsize=(10, 7))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=unique_labels, yticklabels=unique_labels)
        plt.title(f'Confusion Matrix for Participant {base_name}')
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        plt.show()

# Υπολογισμός της μέσης τιμής για κάθε μετρική από όλους τους συμμετέχοντες
bn_mean_accuracy = np.mean(bn_accuracies)
bn_mean_precision = np.mean(bn_precisions)
bn_mean_recall = np.mean(bn_recalls)
bn_mean_f1 = np.mean(bn_f1_scores)

print("Mean Metrics of All Participants:")
print(f"  Mean Accuracy: {bn_mean_accuracy}")
print(f"  Mean Precision: {bn_mean_precision}")
print(f"  Mean Recall: {bn_mean_recall}")
print(f"  Mean F1-Score: {bn_mean_f1}")

**2.4 Σύγκριση Ταξινομητών**

In [ ]:
# Λίστα με τα ονόματα των συμμετεχόντων
participants = [os.path.splitext(filename)[0] for filename in os.listdir(path) if filename.endswith(".csv")]

# Δημιουργία DataFrame για να συγκρίνουμε τις ακριβείες των δύο μοντέλων
accuracy_df = pd.DataFrame({
    'Participant': participants,
    'Random Forest Accuracy': rf_accuracies,
    'Neural Network Accuracy': ann_accuracies,
    'Bayesian Network Accuracy': bn_accuracies
})

# Σχεδιασμός του γραφήματος
plt.figure(figsize=(15, 8))
bar_width = 0.25
index = np.arange(len(participants))

# Μπάρες για τις ακρίβειες του Random Forest
plt.bar(index, accuracy_df['Random Forest Accuracy'][accuracy_df['Participant'] != 'S015'], bar_width, label='Random Forest')

# Μπάρες για τις ακρίβειες του ANN
plt.bar(index + bar_width, accuracy_df['Neural Network Accuracy'][accuracy_df['Participant'] != 'S015'], bar_width, label='ANN')

# Μπάρες για τις ακρίβειες του Bayesian Network
plt.bar(index + 2 * bar_width, accuracy_df['Bayesian Network Accuracy'][accuracy_df['Participant'] != 'S015'], bar_width, label='Bayesian Network')

# Προσθήκη ετικετών και τίτλων
plt.xlabel('Participant')
plt.ylabel('Accuracy')
plt.title('Classifier Accuracy for Each Participant')
plt.xticks(index[accuracy_df['Participant'] != 'S015'] + 1.5 * bar_width, accuracy_df['Participant'][accuracy_df['Participant'] != 'S015'], rotation=45)
plt.legend()

# Εμφάνιση του γραφήματος
plt.tight_layout()
plt.show()